## Import Necessary Libraries

In [ ]:
import gpxpy
import pandas as pd
import cv2
from ultralytics import YOLO
import os
import folium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas 
from reportlab.lib.utils import ImageReader

## Process GPX File Into Pandas Dataframe

In [ ]:
def parseGPX(path):

    gpxFile = open(path, "r") # read the gpx file

    gpx = gpxpy.parse(gpxFile) # parse the gpx xml
    gpxDF = pd.DataFrame(columns=["lat","long","time"]) # create df to hold gpx data
    deltaTime = -1 # variable for the time of gps recording

    # running for the points in the file
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:

                # extract the latitude and longitude
                lat = point.latitude
                long = point.longitude

                if deltaTime == -1: # for the first point set as time zero and save first time point
                    deltaTime = 0
                    date = point.time

                else: # for every other add the time delta and save the next point
                    deltaTime += int((point.time - date).total_seconds())
                    date = point.time

                gpxDF.loc[len(gpxDF)] = [lat,long,deltaTime] # append the results
    gpxDF['flag'] = 0 # add flag for later use
    return gpxDF, date

gpxDF, date = parseGPX("GPX/test.gpx")
print(gpxDF)

          lat       long   time  flag
0   43.005801 -81.279937    0.0     0
1   43.005832 -81.279894    5.0     0
2   43.005860 -81.279898   10.0     0
3   43.005894 -81.279914   15.0     0
4   43.005930 -81.279929   20.0     0
5   43.005961 -81.279943   25.0     0
6   43.005985 -81.279954   30.0     0
7   43.006020 -81.279967   35.0     0
8   43.006049 -81.279980   40.0     0
9   43.006080 -81.279995   45.0     0
10  43.006107 -81.280011   50.0     0
11  43.006137 -81.280026   55.0     0
12  43.006167 -81.280043   60.0     0
13  43.006197 -81.280057   65.0     0
14  43.006229 -81.280074   70.0     0
15  43.006262 -81.280090   75.0     0
16  43.006295 -81.280107   80.0     0
17  43.006328 -81.280126   85.0     0
18  43.006364 -81.280140   90.0     0
19  43.006400 -81.280156   95.0     0
20  43.006441 -81.280173  100.0     0
21  43.006477 -81.280191  105.0     0
22  43.006509 -81.280205  110.0     0
23  43.006542 -81.280211  115.0     0
24  43.006561 -81.280170  120.0     0
25  43.00657

## Process Video and Save Defect Images + Info

In [ ]:
# Load YOLO model
model = YOLO("temp/best_yolo_model.pt")

# Set video paths
source = "temp/test.mp4"
output_folder = r"Frames/"

# Open video
cap = cv2.VideoCapture(source)
fps = 30
frame_count = 0

defectInfo = []

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop when video ends

    # Process every 15th frame
    if frame_count % 15 == 0:
        frame_time = frame_count / fps  # Compute frame timestamp

        # Run inference
        results = model(frame)
        boxes = results[0].boxes
        scores = boxes.conf  # Confidence scores
        labels = boxes.cls  # Class labels
        class_names = ['alligator cracking', 'edge cracking', 'longitudinal cracking', 'patching', 'rutting', 'transverse cracking']

        # Filter detections above 50% confidence
        filtered_boxes = [boxes.xyxy[i].cpu().numpy() for i, score in enumerate(scores) if score > 0.40]


        # If detections exist, annotate and save the frame
        if filtered_boxes:
            annotated_frame = frame.copy()
            for box in filtered_boxes:
                x1, y1, x2, y2 = box
                cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                cv2.putText(annotated_frame, "Defect", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                
                # Get the defect info
                class_confidence = [(class_names[int(label.item())], score.item()) for label, score in zip(labels, scores)]
                sorted_classes = sorted(class_confidence, key=lambda x: x[1], reverse=True)
                sorted_classes = sorted_classes[:3]

                # save as a string
                classed = ", ".join([f"{cls}: {conf * 100:.2f}%" for i, (cls, conf) in enumerate(sorted_classes)])


            # Save the frame as an image
            image_path = f"{output_folder}frame_{frame_count}.jpg"
            cv2.imwrite(image_path, annotated_frame)
            print(f"Saved: {image_path} at {frame_time:.2f}s")

            # Find the closest timestamp in the DataFrame
            closest_idx = gpxDF['time'].sub(frame_time).abs().idxmin()
            closest_time = gpxDF.loc[closest_idx, 'time']
            gpxDF.loc[closest_idx,'flag'] = 1 # set defect flag to true
            defectInfo.append(((gpxDF.loc[closest_idx,'lat'],gpxDF.loc[closest_idx,'long']),classed)) # save defect info
            print(f"Frame time: {frame_time:.2f}s | Closest timestamp: {closest_time:.2f}s")

    frame_count += 1  # Increment frame count

cap.release()



0: 384x640 (no detections), 640.8ms
Speed: 7.0ms preprocess, 640.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 alligator crackings, 394.1ms
Speed: 3.3ms preprocess, 394.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Saved: Frames/frame_15.jpg at 0.50s
Frame time: 0.50s | Closest timestamp: 0.00s

0: 384x640 (no detections), 441.6ms
Speed: 12.1ms preprocess, 441.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 347.0ms
Speed: 2.8ms preprocess, 347.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 543.7ms
Speed: 1.9ms preprocess, 543.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 356.6ms
Speed: 2.0ms preprocess, 356.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 341.6ms
Speed: 3.3ms preprocess, 341.6ms inference, 0.6ms postprocess per

## Get Marked Up Satellite Map

In [ ]:
def plotMap(coords, zoom_start=20):
   
    # Get Center Point for Map
    avg_lat = sum(lat for lat, lon in coords) / len(coords)
    avg_lon = sum(lon for lat, lon in coords) / len(coords)

    # Create folium map and using argisonline to get sat image
    m = folium.Map(
    location=[avg_lat, avg_lon], 
    zoom_start=zoom_start,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri"
    )

    # Extract Coordinates and Defect Flag to Create Map
    for coord, flag in zip(coords,gpxDF['flag']):
        if flag == 0: # No Defect = Blue
            folium.Marker([coord[0], coord[1]]).add_to(m)
        else: # Defect = Red
            folium.Marker([coord[0], coord[1]],icon=folium.Icon(color="red")).add_to(m)

    # Save map as an HTML file
    map_file = "Maps/map.html"
    m.save(map_file)

    # Convert relative path to absolute
    map_path = os.path.abspath(map_file)
    map_url = f"file:///{map_path}"

    # Setup Selenium
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)
    driver.set_window_size(800, 600)
    driver.get(map_url)
    time.sleep(2)  # Wait for the map to load

    # Save screenshot
    mapPic_path = "Maps/map.png"
    driver.save_screenshot(mapPic_path)
    driver.quit()

coords = list(zip(gpxDF['lat'],gpxDF['long']))
plotMap(coords)

## Generate Final PDF Report

In [ ]:
def create_pdf(output_filename, map_image_path, images_directory):
    c = canvas.Canvas(output_filename, pagesize=A4) # Set to typical page
    width, height = A4  # A4 dimensions

    # Title
    c.setFont("Helvetica-Bold", 20)
    c.drawString(100, height - 50, "Pavement Damage Report")

    # Lines for test information
    c.setFont("Helvetica", 12)
    y_position = height - 80
    c.drawString(100, y_position, f"Date: {date.date()}")
    y_position -= 20
    c.drawString(100, y_position, "Location: Springett Staff Lot (Rear)")
    y_position -= 20
    c.drawString(100, y_position, f"Runtime: {gpxDF.loc[gpxDF.index[-1], 'time']} seconds")
    y_position -= 20
    c.drawString(100, y_position, f"Defects Counted: {sum(gpxDF['flag'])}")
    y_position -= 20
    c.drawString(100, y_position, f"Percent Damaged: {round(sum(gpxDF['flag']/len(gpxDF)*100),-1)}%")
    y_position -= 20

    # Subtitle for the map
    c.setFont("Helvetica-Bold", 16)
    y_position -= 40
    c.drawString(100, y_position, "Map Overview")

    # Add the map image
    y_position -= 250  # Space for the image
    c.drawImage(ImageReader(map_image_path), 100, y_position, width=400, height=250, preserveAspectRatio=True)

    # Subtitle for defect images section
    y_position -= 40
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, y_position, "Captured Images")

    # Iterate through defect images directory
    y_position -= 40
    x=1
    for img_file in sorted(os.listdir(images_directory)):
        img_path = os.path.join(images_directory, img_file)

        if y_position < 250:  # Create a new page if space is low
            c.showPage()
            c.setFont("Helvetica-Bold", 16)
            y_position = height - 50  # Reset position
            
        c.setFont("Helvetica", 12)
            
        # Write defect image info
        c.drawString(100, y_position, f"Defect Number: {x}")
        y_position -= 20
        c.drawString(100, y_position, f"Defect Position: {defectInfo[x-1][0]}")
        y_position -= 20
        c.drawString(100, y_position, f"Suspected Defects: {defectInfo[x-1][1]}")
        y_position -= 20

        # Adjust y position before adding the image
        y_position -= 200  

        # Add image
        c.drawImage(ImageReader(img_path), 100, y_position, width=400, height=200, preserveAspectRatio=True)

        # Adjust y position after the image
        y_position -= 20

        x += 1


    c.save()

create_pdf("pavement_report.pdf", "Maps/map.png", "Frames/")
